In [3]:
import pandas as pd
import pysam
import copy
from Bio.Seq import Seq

In [21]:
rp = pd.read_csv('/SGRNJ06/randd/USER/cjj/TESTDATA/test_trust/20220607refactor/fltest/03.summarize/fltest_t.csv')

In [22]:
rp['productive'] = rp['full_length']
#df['productive'] = df['cdr3'].apply(lambda x: False if x=='None' else True)

In [23]:
rp

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis,raw_clonotype_id,raw_consensus_id
0,GAGCTGAACCGTGAGAGCTAACGA,True,GAGCTGAACCGTGAGAGCTAACGA_826,True,None,TRB,TRBV25-1*01,None,TRBJ1-1*01,TRBC1,False,False,CASSEYGEAFF,TGTGCCAGCAGTGAATATGGTGAAGCTTTCTTT,1.0,1.0,None,None
1,AATCCGTCAATCCGTCCCGTGAGA,True,AATCCGTCAATCCGTCCCGTGAGA_1023,True,None,TRB,TRBV2*01,TRBD2*01,TRBJ1-1*01,None,False,False,CAISASGWTEYFF,TGTGCCATCAGCGCCAGTGGTTGGACTGAATATTTCTTT,1.0,1.0,None,None
2,AACTCACCGACAGTGCAGTGGTCA,True,AACTCACCGACAGTGCAGTGGTCA_1052,True,None,TRB,TRBV19*01,None,TRBJ2-7*01,TRBC2,False,False,CACSMLATGYEEYL,TGTGCCTGTAGTATGTTGGCTACCGGCTACGAGGAGTACTTG,1.0,1.0,None,None
3,AATCCGTCGAGCTGAAAAACATCG,True,AATCCGTCGAGCTGAAAAACATCG_315,True,None,TRB,TRBV2*01,TRBD2*01,TRBJ2-7*01,TRBC2,False,False,CASTAGRSYEQYF,TGTGCCAGCACTGCGGGCAGGTCCTACGAGCAGTACTTC,1.0,1.0,None,None
4,CCTCCTGACGCATACACGCTGATC,True,CCTCCTGACGCATACACGCTGATC_951,True,None,TRA,TRAV1-2*01,None,TRAJ33*01,TRAC,False,False,CADCDSNDQLIW,TGTGCTGATTGTGATAGCAACGATCAGTTAATCTGG,1.0,1.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,AGAGTCAAAAGAGATCTATCAGCA,True,AGAGTCAAAAGAGATCTATCAGCA_989,True,None,TRB,TRBV25-1*01,None,TRBJ1-1*01,None,False,False,CASSEYGEAFF,TGTGCCAGCAGTGAATATGGTGAAGCTTTCTTT,1.0,1.0,None,None
528,ATTGAGGACACCTTACCTGGCATA,True,ATTGAGGACACCTTACCTGGCATA_351,True,None,TRA,TRAV1-2*01,None,TRAJ33*01,TRAC,False,False,CAACDSNYQLIW,TGTGCTGCTTGTGATAGCAACTATCAGTTAATCTGG,1.0,1.0,None,None
529,CGGATTGCAAGGACACATCATTCC,True,CGGATTGCAAGGACACATCATTCC_483,True,None,TRB,TRBV2*01,TRBD1*01,TRBJ1-1*01,TRBC1,False,False,CASSARGWTEAFF,TGTGCCAGCAGCGCCAGGGGTTGGACTGAAGCTTTCTTT,1.0,1.0,None,None
530,CTGGCATACGCATACACCTCCTGA,True,CTGGCATACGCATACACCTCCTGA_1175,True,None,TRA,TRAV13-1*01,None,TRAJ36*01,TRAC,False,False,CAESFQTGANNLFF,TGTGCAGAAAGCTTTCAAACTGGGGCAAACAACCTCTTCTTT,1.0,1.0,None,None


In [24]:
        rp.sort_values(by='umis', ascending=False, inplace=True)

        df_chain_heavy = rp[rp['chain']=='TRA']
        df_chain_light = rp[(rp['chain']=='TRB')]

        df_chain_heavy = df_chain_heavy.drop_duplicates(['barcode'])
        df_chain_light = df_chain_light.drop_duplicates(['barcode'])
        df_for_clono = pd.concat([df_chain_heavy, df_chain_light], ignore_index=True)

In [25]:
df_for_clono

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis,raw_clonotype_id,raw_consensus_id
0,CGCTGATCCCGAAGTAACGCTCGA,True,CGCTGATCCCGAAGTAACGCTCGA_58,True,None,TRA,TRAV13-1*02,None,TRAJ36*01,TRAC,True,True,CAASFQTGANNLFF,TGTGCAGCAAGCTTTCAAACTGGGGCAAACAACCTCTTCTTT,10.0,10.0,None,None
1,CCAGTTCAAGTCACTAAGATGTAC,True,CCAGTTCAAGTCACTAAGATGTAC_73,True,None,TRA,TRAV13-1*02,None,TRAJ36*01,TRAC,True,True,CAASFQTGANNLFF,TGTGCAGCAAGCTTTCAAACTGGGGCAAACAACCTCTTCTTT,9.0,9.0,None,None
2,GCTAACGAGTGTTCTAACATTGGC,True,GCTAACGAGTGTTCTAACATTGGC_235,True,None,TRA,TRAV13-1*01,None,TRAJ36*01,TRAC,False,False,CAASFQTGANNLFF,TGTGCAGCAAGCTTTCAAACTGGGGCAAACAACCTCTTCTTT,7.0,7.0,None,None
3,GGAGAACACCTCCTGAATGCCTAA,True,GGAGAACACCTCCTGAATGCCTAA_38,True,None,TRA,TRAV13-1*02,None,TRAJ36*01,TRAC,True,True,CAASFQTGANNLFF,TGTGCAGCAAGCTTTCAAACTGGGGCAAACAACCTCTTCTTT,6.0,6.0,None,None
4,CATACCAATGGAACAACCAGTTCA,True,CATACCAATGGAACAACCAGTTCA_165,True,None,TRA,TRAV1-2*01,None,TRAJ33*01,TRAC,True,True,CAACDSNYQLIW,TGTGCTGCTTGTGATAGCAACTATCAGTTAATCTGG,6.0,6.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,GATAGACAATCCTGTACATCAAGT,True,GATAGACAATCCTGTACATCAAGT_799,True,None,TRB,TRBV2*01,TRBD1*01,TRBJ1-1*01,TRBC1,False,False,CASSARGWTEAFF,TGTGCCAGCAGCGCCAGGGGTTGGACTGAAGCTTTCTTT,1.0,1.0,None,None
528,CACTTCGACGGATTGCAGCCATGC,True,CACTTCGACGGATTGCAGCCATGC_263,True,None,TRB,TRBV28*01,TRBD2*02,TRBJ2-3*01,None,False,False,CASSSIAGEDADTQYF,TGTGCCAGCAGTTCCATAGCGGGAGAAGACGCAGATACGCAGTATTTT,1.0,1.0,None,None
529,AAGGACACAACTCACCCCTAATCC,True,AAGGACACAACTCACCCCTAATCC_800,True,None,TRB,TRBV28*01,TRBD2*02,TRBJ2-3*01,TRBC2,False,False,CASSSIAGEDADTQYF,TGTGCCAGCAGTTCCATAGCGGGAGAAGACGCAGATACGCAGTATTTT,1.0,1.0,None,None
530,AGAGTCAACACCTTACCTAAGGTC,True,AGAGTCAACACCTTACCTAAGGTC_1151,True,None,TRB,TRBV18*01,None,TRBJ2-3*01,TRBC2,False,False,CERSLSSDSQYF,TGTGAAAGATCTCTGTCCTCAGATTCGCAGTATTTT,1.0,1.0,None,None


,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,cdr3,cdr3_nt,reads,umis,raw_clonotype_id,raw_consensus_id
productive,,,,,,,,,,,,,,,,,
False,2984,2984,2984,2984,2984,2984,2984,2984,2984,2984,2984,2984,2984,2984,2984,2984,2984
True,9454,9454,9454,9454,9454,9454,9454,9454,9454,9454,9454,9454,9454,9454,9454,9454,9454


In [ ]:
rp = rp[rp['full_length']==True]

In [ ]:
rp

In [ ]:
rp1 = copy.deepcopy(rp)

In [ ]:
rp1['cdr3'] = rp1['cdr3_nt'].apply(lambda x: 'None' if "*" in str(Seq(x).translate()) or not len(x)%3==0 else str(Seq(x).translate()))

In [ ]:
rp1.cdr3.tolist() == rp.cdr3.tolist()

In [158]:
len_dict = dict()
with pysam.FastxFile('/SGRNJ06/randd/USER/cjj/celedev/kemai/20220512mergeTR/sc-1/03.assemble/assemble/sc-1_annot.fa') as fa:
    for read in fa:
        len_dict[read.name] = read.comment.split(' ')[0]

In [178]:
len_dict['TTCTGTGTTTGGTATGGAATCTGT_135058']

'190'

In [173]:
rp['length'] = rp['contig_id'].apply(lambda x: len_dict[x])

In [174]:
rp

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis,raw_clonotype_id,raw_consensus_id
0,TCATTGAGTCCTCAATTGTACCTT,True,TCATTGAGTCCTCAATTGTACCTT_19898,True,497,IGH,IGHV4-39*01,IGHD4-17*01,IGHJ4*02,IGHG2B*03,True,True,CARPSNWGSDYFDYW,TGTGCGAGACCCTCTAACTGGGGATCCGACTACTTTGACTACTGG,9.0,9.0,None,None
1,TCATTGAGTCCTCAATTGTACCTT,True,TCATTGAGTCCTCAATTGTACCTT_2152,True,524,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,102.0,102.0,None,None
2,TGCGTGAATGTGAAGATATGCCAG,True,TGCGTGAATGTGAAGATATGCCAG_228274,True,244,IGH,IGHV3-48*03,IGHD1-7*01,IGHJ2*01,IGHG1*01,False,False,CARDTWNYAKWYFDLW,TGTGCGAGAGATACCTGGAACTACGCTAAGTGGTACTTCGATCTCTGG,21.0,21.0,None,None
3,TGCGTGAATGTGAAGATATGCCAG,True,TGCGTGAATGTGAAGATATGCCAG_759,True,578,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,553.0,553.0,None,None
4,TATGTGGCGACCTTAGTGCGTGAA,True,TATGTGGCGACCTTAGTGCGTGAA_307739,True,231,IGH,IGHV6-1*01,IGHD4-23*01,IGHJ3*02,IGHA*09,False,False,CARERGDYGGNDAFDIW,TGTGCAAGAGAGAGGGGGGACTACGGTGGTAACGATGCTTTTGATA...,5.0,5.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12433,GAATCTGTTGGCTCAGTACTTCGG,True,GAATCTGTTGGCTCAGTACTTCGG_3087,True,519,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,33.0,33.0,None,None
12434,TTGGAGGTTGAACTGGTGCGATCT,True,TTGGAGGTTGAACTGGTGCGATCT_13339,True,430,IGH,IGHV3-20*01,IGHD1-20*01,IGHJ4*02,IGHG2C*03,True,True,CARGITGNDYW,TGTGCGAGGGGTATAACTGGAAATGACTACTGG,12.0,12.0,None,None
12435,TTGGAGGTTGAACTGGTGCGATCT,True,TTGGAGGTTGAACTGGTGCGATCT_1614,True,524,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,177.0,177.0,None,None
12436,TTCTGTGTTTGGTATGGAATCTGT,True,TTCTGTGTTTGGTATGGAATCTGT_135058,True,190,IGH,IGHV3-48*03,IGHD3-3*01,IGHJ4*02,IGHG2B*03,False,False,CARGADYW,TGTGCGAGAGGGGCTGACTACTGG,9.0,9.0,None,None


In [163]:
rp

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis,raw_clonotype_id,raw_consensus_id
0,TCATTGAGTCCTCAATTGTACCTT,True,TCATTGAGTCCTCAATTGTACCTT_19898,True,None,IGH,IGHV4-39*01,IGHD4-17*01,IGHJ4*02,IGHG2B*03,True,True,CARPSNWGSDYFDYW,TGTGCGAGACCCTCTAACTGGGGATCCGACTACTTTGACTACTGG,9.0,9.0,None,None
1,TCATTGAGTCCTCAATTGTACCTT,True,TCATTGAGTCCTCAATTGTACCTT_2152,True,None,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,102.0,102.0,None,None
2,TGCGTGAATGTGAAGATATGCCAG,True,TGCGTGAATGTGAAGATATGCCAG_228274,True,None,IGH,IGHV3-48*03,IGHD1-7*01,IGHJ2*01,IGHG1*01,False,False,CARDTWNYAKWYFDLW,TGTGCGAGAGATACCTGGAACTACGCTAAGTGGTACTTCGATCTCTGG,21.0,21.0,None,None
3,TGCGTGAATGTGAAGATATGCCAG,True,TGCGTGAATGTGAAGATATGCCAG_759,True,None,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,553.0,553.0,None,None
4,TATGTGGCGACCTTAGTGCGTGAA,True,TATGTGGCGACCTTAGTGCGTGAA_307739,True,None,IGH,IGHV6-1*01,IGHD4-23*01,IGHJ3*02,IGHA*09,False,False,CARERGDYGGNDAFDIW,TGTGCAAGAGAGAGGGGGGACTACGGTGGTAACGATGCTTTTGATA...,5.0,5.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12433,GAATCTGTTGGCTCAGTACTTCGG,True,GAATCTGTTGGCTCAGTACTTCGG_3087,True,None,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,33.0,33.0,None,None
12434,TTGGAGGTTGAACTGGTGCGATCT,True,TTGGAGGTTGAACTGGTGCGATCT_13339,True,None,IGH,IGHV3-20*01,IGHD1-20*01,IGHJ4*02,IGHG2C*03,True,True,CARGITGNDYW,TGTGCGAGGGGTATAACTGGAAATGACTACTGG,12.0,12.0,None,None
12435,TTGGAGGTTGAACTGGTGCGATCT,True,TTGGAGGTTGAACTGGTGCGATCT_1614,True,None,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,177.0,177.0,None,None
12436,TTCTGTGTTTGGTATGGAATCTGT,True,TTCTGTGTTTGGTATGGAATCTGT_135058,True,None,IGH,IGHV3-48*03,IGHD3-3*01,IGHJ4*02,IGHG2B*03,False,False,CARGADYW,TGTGCGAGAGGGGCTGACTACTGG,9.0,9.0,None,None


In [157]:
for i in set(rp.contig_id):
    rp.length = len_dict[i]

KeyError: 'TCTCGGTTTGATACGTTGTCTATC_1681'

In [165]:
rp.length = '1'

In [166]:
rp

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis,raw_clonotype_id,raw_consensus_id
0,TCATTGAGTCCTCAATTGTACCTT,True,TCATTGAGTCCTCAATTGTACCTT_19898,True,1,IGH,IGHV4-39*01,IGHD4-17*01,IGHJ4*02,IGHG2B*03,True,True,CARPSNWGSDYFDYW,TGTGCGAGACCCTCTAACTGGGGATCCGACTACTTTGACTACTGG,9.0,9.0,None,None
1,TCATTGAGTCCTCAATTGTACCTT,True,TCATTGAGTCCTCAATTGTACCTT_2152,True,1,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,102.0,102.0,None,None
2,TGCGTGAATGTGAAGATATGCCAG,True,TGCGTGAATGTGAAGATATGCCAG_228274,True,1,IGH,IGHV3-48*03,IGHD1-7*01,IGHJ2*01,IGHG1*01,False,False,CARDTWNYAKWYFDLW,TGTGCGAGAGATACCTGGAACTACGCTAAGTGGTACTTCGATCTCTGG,21.0,21.0,None,None
3,TGCGTGAATGTGAAGATATGCCAG,True,TGCGTGAATGTGAAGATATGCCAG_759,True,1,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,553.0,553.0,None,None
4,TATGTGGCGACCTTAGTGCGTGAA,True,TATGTGGCGACCTTAGTGCGTGAA_307739,True,1,IGH,IGHV6-1*01,IGHD4-23*01,IGHJ3*02,IGHA*09,False,False,CARERGDYGGNDAFDIW,TGTGCAAGAGAGAGGGGGGACTACGGTGGTAACGATGCTTTTGATA...,5.0,5.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12433,GAATCTGTTGGCTCAGTACTTCGG,True,GAATCTGTTGGCTCAGTACTTCGG_3087,True,1,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,33.0,33.0,None,None
12434,TTGGAGGTTGAACTGGTGCGATCT,True,TTGGAGGTTGAACTGGTGCGATCT_13339,True,1,IGH,IGHV3-20*01,IGHD1-20*01,IGHJ4*02,IGHG2C*03,True,True,CARGITGNDYW,TGTGCGAGGGGTATAACTGGAAATGACTACTGG,12.0,12.0,None,None
12435,TTGGAGGTTGAACTGGTGCGATCT,True,TTGGAGGTTGAACTGGTGCGATCT_1614,True,1,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,177.0,177.0,None,None
12436,TTCTGTGTTTGGTATGGAATCTGT,True,TTCTGTGTTTGGTATGGAATCTGT_135058,True,1,IGH,IGHV3-48*03,IGHD3-3*01,IGHJ4*02,IGHG2B*03,False,False,CARGADYW,TGTGCGAGAGGGGCTGACTACTGG,9.0,9.0,None,None


In [63]:
a = pd.read_csv('/SGRNJ06/randd/USER/cjj/TESTDATA/test_trust/20220607refactor/fltest/03.summarize/fltest_all_contig.csv')

In [66]:
a

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis,clonotype_id
0,CCAGTTCAAGTCACTAAGATGTAC,True,CCAGTTCAAGTCACTAAGATGTAC_2,True,502,TRB,TRBV2*01,TRBD1*01,TRBJ1-1*01,TRBC1,True,True,CASSARGWTEAFF,TGTGCCAGCAGCGCCAGGGGTTGGACTGAAGCTTTCTTT,17.0,17.0,clonotype1
1,CCAGTTCAAGTCACTAAGATGTAC,True,CCAGTTCAAGTCACTAAGATGTAC_73,True,462,TRA,TRAV13-1*02,None,TRAJ36*01,TRAC,True,True,CAASFQTGANNLFF,TGTGCAGCAAGCTTTCAAACTGGGGCAAACAACCTCTTCTTT,9.0,9.0,clonotype1
2,CGCTGATCCCGAAGTAACGCTCGA,True,CGCTGATCCCGAAGTAACGCTCGA_0,True,541,TRB,TRBV2*01,TRBD1*01,TRBJ1-1*01,TRBC1,True,True,CASSARGWTEAFF,TGTGCCAGCAGCGCCAGGGGTTGGACTGAAGCTTTCTTT,27.0,27.0,clonotype1
3,CGCTGATCCCGAAGTAACGCTCGA,True,CGCTGATCCCGAAGTAACGCTCGA_58,True,567,TRA,TRAV13-1*02,None,TRAJ36*01,TRAC,True,True,CAASFQTGANNLFF,TGTGCAGCAAGCTTTCAAACTGGGGCAAACAACCTCTTCTTT,10.0,10.0,clonotype1
4,AACCGAGAGCCACATAGTCGTAGA,True,AACCGAGAGCCACATAGTCGTAGA_9,True,511,TRB,TRBV2*01,TRBD1*01,TRBJ1-1*01,TRBC1,True,True,CASSARGWTEAFF,TGTGCCAGCAGCGCCAGGGGTTGGACTGAAGCTTTCTTT,3.0,3.0,clonotype1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,ACAGCAGAAACTCACCAGAGTCAA,True,ACAGCAGAAACTCACCAGAGTCAA_510,True,150,TRB,TRBV2*01,TRBD1*01,TRBJ1-1*01,None,False,False,CASSARGWTEAFF,TGTGCCAGCAGCGCCAGGGGTTGGACTGAAGCTTTCTTT,1.0,1.0,None
528,ACACAGAACGGATTGCCTGGCATA,True,ACACAGAACGGATTGCCTGGCATA_285,True,150,TRB,TRBV28*01,TRBD2*02,TRBJ2-3*01,TRBC2,False,False,CASSSIAGEDADTQYF,TGTGCCAGCAGTTCCATAGCGGGAGAAGACGCAGATACGCAGTATTTT,1.0,1.0,None
529,TAGGATGAGAATCTGATGAAGAGA,True,TAGGATGAGAATCTGATGAAGAGA_44,True,286,TRB,TRBV2*01,TRBD1*01,TRBJ1-1*01,TRBC1,False,False,CASSARGWTEAFF,TGTGCCAGCAGCGCCAGGGGTTGGACTGAAGCTTTCTTT,1.0,1.0,None
530,AACTCACCCGACACACATCCTGTA,True,AACTCACCCGACACACATCCTGTA_1286,True,149,TRA,TRAV38-2/DV8*01,None,TRAJ42*01,TRAC,False,False,CAGGTGRLGILIF,TGTGCTGGGGGGACTGGTAGGCTAGGAATTCTCATCTTT,1.0,1.0,None


In [62]:
get_vj_annot(ten, ['TRA', 'TRB'], ['TRA_TRB'])

[{'name': 'Cells With Productive V-J Spanning Pair',
  'value': 6626,
  'total': 8203},
 {'name': 'Cells With Productive V-J Spanning (TRA, TRB) Pair',
  'value': 3779,
  'total': 8203,
  'help_info': "Fraction of cell-associated barcodes with one productive contig for each chain of the receptor pair.A productive contig satisfies the following conditions: the contig annotations span the 5' end of the V region to the 3' end of the J region of the chain, a start codon was found in the expected part of the V sequence, an in-frame CDR3 amino acid motif was found, and no stop codons were found in the aligned V-J region"},
 {'name': 'Cells With TRA Contig',
  'value': 5004,
  'total': 8203,
  'help_info': 'Fraction of cell-associated barcodes with at least one TRA contig annotated as a full or partial V(D)J gene'},
 {'name': 'Cells With CDR3-annotated TRA Contig',
  'value': 3991,
  'total': 8203},
 {'name': 'Cells With V-J Spanning TRA Contig',
  'value': 4630,
  'total': 8203,
  'help_info

In [61]:
def get_vj_annot(df, chains, pairs):
    fl_pro_pair_df = pd.DataFrame(df[df['productive']==True].barcode.value_counts())
    fl_pro_pair_df = fl_pro_pair_df[fl_pro_pair_df['barcode']>=2]
    Result = []
    cell_nums = len(set(df['barcode'].tolist()))
    Result.append({
        'name': 'Cells With Productive V-J Spanning Pair',
        'value': fl_pro_pair_df.shape[0],
        'total': cell_nums,
    })
    for p in pairs:
        chain1 = p.split('_')[0]
        chain2 = p.split('_')[1]
        cbs1 = set(df[(df['full_length']==True)&(df['productive']==True)&(df['chain']==chain1)].barcode.tolist())
        cbs2 = set(df[(df['full_length']==True)&(df['productive']==True)&(df['chain']==chain2)].barcode.tolist())
        paired_cbs = len(cbs1.intersection(cbs2))
        Result.append({
            'name': f'Cells With Productive V-J Spanning ({chain1}, {chain2}) Pair',
            'value': paired_cbs,
            'total': cell_nums,
            'help_info': "Fraction of cell-associated barcodes with one productive contig for each chain of the receptor pair.A productive contig satisfies the following conditions: the contig annotations span the 5' end of the V region to the 3' end of the J region of the chain, a start codon was found in the expected part of the V sequence, an in-frame CDR3 amino acid motif was found, and no stop codons were found in the aligned V-J region"
        })
    for c in chains:
        Result.append({
            'name': f'Cells With {c} Contig',
            'value': len(set(df[df['chain']==c].barcode.tolist())),
            'total': cell_nums,
            'help_info': f'Fraction of cell-associated barcodes with at least one {c} contig annotated as a full or partial V(D)J gene'
        })
        Result.append({
            'name': f'Cells With CDR3-annotated {c} Contig',
            'value': len(set(df[(df['chain']==c)&(df['productive']==True)].barcode.tolist())),
            'total': cell_nums,
        })
        Result.append({
            'name': f'Cells With V-J Spanning {c} Contig',
            'value': len(set(df[(df['full_length']==True)&(df['chain']==c)].barcode.tolist())),
            'total': cell_nums,
            'help_info': f"Fraction of cell-associated barcodes with at least one contig spanning the 5' end of the V region to the 3' end of the J region for {c}"
        })
        Result.append({
            'name': f'Cells With Productive {c} Contig',
            'value': len(set(df[(df['full_length']==True)&(df['productive']==True)&(df['chain']==c)].barcode.tolist())),
            'total': cell_nums,
            'help_info': "Fraction of cell-associated barcodes with productive IGL chain. A productive contig satisfies the following conditions: the contig annotations span the 5' end of the V region to the 3' end of the J region of the chain, a start codon was found in the expected part of the V sequence, an in-frame CDR3 amino acid motif was found, and no stop codons were found in the aligned V-J region"
        })

    return Result

In [ ]:
get_vj_annot(rp, ['IGH', 'IGL', 'IGK'], ['IGH_IGL', 'IGH_IGK'])

In [73]:
get_vj_annot(ten, ['TRA', 'TRB'], ['TRA_TRB'])

[{'name': 'Cells With Productive V-J Spanning Pair',
  'value': 6626,
  'total': 8203},
 {'name': 'Cells With Productive V-J Spanning (TRA, TRB) Pair',
  'value': 3779,
  'total': 8203,
  'help_info': "Fraction of cell-associated barcodes with one productive contig for each chain of the receptor pair.A productive contig satisfies the following conditions: the contig annotations span the 5' end of the V region to the 3' end of the J region of the chain, a start codon was found in the expected part of the V sequence, an in-frame CDR3 amino acid motif was found, and no stop codons were found in the aligned V-J region"},
 {'name': 'Cells With TRA Contig',
  'value': 5004,
  'total': 8203,
  'help_info': 'Fraction of cell-associated barcodes with at least one TRA contig annotated as a full or partial V(D)J gene'},
 {'name': 'Cells With CDR3-annotated TRA Contig',
  'value': 3991,
  'total': 8203},
 {'name': 'Cells With V-J Spanning TRA Contig',
  'value': 4630,
  'total': 8203,
  'help_info

In [69]:
ten = pd.read_csv('/SGRNJ06/randd/PROJECT/RD20073101_ScRNA_VDJ/20220516/H_0421PBMC_1_NK_3TNlib/03.assemble/H_0421PBMC_1_NK_3TNlib/outs/filtered_contig_annotations.csv')

In [70]:
len(set(ten['barcode']))

8203

In [71]:
ten.barcode = ten.barcode.apply(lambda x:x.split('-')[0])

In [72]:
ten

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis,raw_clonotype_id,raw_consensus_id
0,AAACCTGAGAAACCTA,True,AAACCTGAGAAACCTA-1_contig_1,True,702,TRA,TRAV21,None,TRAJ37,TRAC,True,True,CAVTNSGNTGKLIF,TGTGCTGTGACGAACTCTGGCAACACAGGCAAACTAATCTTT,3511,16,clonotype256,clonotype256_consensus_2
1,AAACCTGAGAAACCTA,True,AAACCTGAGAAACCTA-1_contig_2,True,725,TRB,TRBV7-8,None,TRBJ2-7,TRBC2,True,True,CASSLMRGVSYEQYF,TGTGCCAGCAGCTTGATGCGGGGGGTCTCCTACGAGCAGTACTTC,2082,31,clonotype256,clonotype256_consensus_1
2,AAACCTGAGAAACCTA,True,AAACCTGAGAAACCTA-1_contig_3,True,440,TRB,None,None,TRBJ2-2P,TRBC2,False,False,None,None,19,3,clonotype256,None
3,AAACCTGAGAAACCTA,True,AAACCTGAGAAACCTA-1_contig_4,True,699,TRA,TRAV36/DV7,None,TRAJ52,TRAC,True,False,None,None,115,10,clonotype256,None
4,AAACCTGAGAAACCTA,True,AAACCTGAGAAACCTA-1_contig_5,True,307,TRB,TRBV7-2,None,TRBJ2-7,TRBC2,True,False,CASSLMRGVSYEQYF,TGTGCCAGCAGCTTGATGCGGGGGGTCTCCTACGAGCAGTACTTC,32,6,clonotype256,clonotype256_consensus_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38701,CTCTGGTGTACAGTTC,True,CTCTGGTGTACAGTTC-1_contig_2,True,483,TRB,TRBV25-1,None,TRBJ2-7,TRBC2,True,True,CAISVAGRDEQYL,TGTGCCATCAGTGTCGCGGGACGGGACGAGCAGTACTTG,28,1,clonotype6200,clonotype6200_consensus_2
38702,CTTACCGTCTGGTGTA,True,CTTACCGTCTGGTGTA-1_contig_1,True,457,TRA,TRAV12-3,None,TRAJ23,TRAC,True,True,CAMSEWYNQGGKLIF,TGTGCAATGAGCGAATGGTATAACCAGGGAGGAAAGCTTATCTTC,10,1,clonotype6201,clonotype6201_consensus_1
38703,CTTACCGTCTGGTGTA,True,CTTACCGTCTGGTGTA-1_contig_2,True,486,TRB,TRBV7-9,None,TRBJ2-2,TRBC2,True,True,CASSLAGRGTGELFF,TGTGCCAGCAGCTTAGCCGGGAGGGGCACCGGGGAGCTGTTTTTT,21,4,clonotype6201,clonotype6201_consensus_2
38704,GATCGCGTCTCGCATC,True,GATCGCGTCTCGCATC-1_contig_1,True,455,TRB,TRBV24-1,None,TRBJ2-1,TRBC2,True,True,CATASGSYNEQFF,TGTGCCACCGCTAGCGGGAGCTACAATGAGCAGTTCTTC,21,1,clonotype6202,clonotype6202_consensus_2


In [188]:
ten[ten['productive']==True]

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis,raw_clonotype_id,raw_consensus_id
0,AAACCTGAGAAACCTA-1,True,AAACCTGAGAAACCTA-1_contig_1,True,487,TRB,TRBV3-1,None,TRBJ1-1,TRBC1,True,True,CASSLNTEAFF,TGTGCCAGCAGCCTGAACACTGAAGCTTTCTTT,5585,10,clonotype212,clonotype212_consensus_1
3,AAACCTGAGAAACCTA-1,True,AAACCTGAGAAACCTA-1_contig_2,True,750,TRA,TRAV26-1,None,TRAJ49,TRAC,True,True,CIVIAYSTGNQFYF,TGCATCGTCATTGCGTATAGCACCGGTAACCAGTTCTATTTT,1699,11,clonotype212,clonotype212_consensus_2
11,AAACCTGAGAAACGAG-1,True,AAACCTGAGAAACGAG-1_contig_1,True,472,TRB,TRBV28,None,TRBJ1-4,TRBC1,True,True,CASSLWQGTNEKLFF,TGTGCCAGCAGTTTATGGCAGGGAACTAATGAAAAACTGTTTTTT,6781,19,clonotype12,clonotype12_consensus_2
16,AAACCTGAGAAACGAG-1,True,AAACCTGAGAAACGAG-1_contig_2,True,707,TRA,TRAV36/DV7,None,TRAJ28,TRAC,True,True,CAAPAAGSYQLTF,TGTGCTGCGCCCGCCGCTGGGAGTTACCAACTCACTTTC,2006,11,clonotype12,clonotype12_consensus_1
24,AAACCTGAGAAACGCC-1,True,AAACCTGAGAAACGCC-1_contig_1,True,481,TRB,TRBV4-1,None,TRBJ2-2,TRBC2,True,True,CASSPGARGNGELFF,TGCGCCAGCAGCCCCGGAGCCCGGGGAAACGGGGAGCTGTTTTTT,107,2,clonotype213,clonotype213_consensus_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37099,CGTAGCGAGCCGGTAA-1,True,CGTAGCGAGCCGGTAA-1_contig_2,True,449,TRB,TRBV29-1,None,TRBJ2-1,TRBC2,True,True,CSVEVWDSSYNEQFF,TGCAGCGTTGAAGTTTGGGACAGTTCCTACAATGAGCAGTTCTTC,6,1,clonotype4899,clonotype4899_consensus_2
37100,CGTTCTGCATCTCGCT-1,True,CGTTCTGCATCTCGCT-1_contig_1,True,472,TRB,TRBV28,None,TRBJ1-4,TRBC1,True,True,CASSLWSGSNEKLFF,TGTGCCAGCAGTTTATGGTCAGGGTCGAACGAAAAACTGTTTTTT,24,3,clonotype2,clonotype2_consensus_1
37101,CGTTCTGCATCTCGCT-1,True,CGTTCTGCATCTCGCT-1_contig_2,True,630,TRA,TRAV36/DV7,None,TRAJ28,TRAC,True,True,CAAPGAGSYQLTF,TGTGCTGCCCCTGGGGCTGGGAGTTACCAACTCACTTTC,35,4,clonotype2,clonotype2_consensus_2
37102,GAATAAGCAGCTGTAT-1,True,GAATAAGCAGCTGTAT-1_contig_1,True,457,TRA,TRAV26-2,None,TRAJ17,TRAC,True,True,CILRDVGAAGNKLTF,TGCATCCTGAGAGACGTCGGAGCTGCAGGCAACAAGCTAACTTTT,16,1,clonotype4900,clonotype4900_consensus_1
